In [1]:
import numpy as np
import pandas as pd
import os
import sys
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

In [2]:
df = pd.read_pickle('gradient_boosting_data.pkl')
df_accent_mfccs = df.drop(['filename', 'sex'], axis=1)
df.head()

,filename,accent,sex,mfccs0,mfccs1,mfccs2,mfccs3,mfccs4,mfccs5,mfccs6,...,mfccs10,mfccs11,mfccs12,mfccs13,mfccs14,mfccs15,mfccs16,mfccs17,mfccs18,mfccs19
0,/home/andriy/diploma/data/p253/p253_157.wav,Welsh,F,-296.041776,51.941099,0.088696,50.687054,9.213817,-12.981535,1.850823,...,-6.183682,7.018186,-0.891467,1.275083,-1.895707,-4.575062,-2.737582,2.220370,-0.677952,-0.549451
1,/home/andriy/diploma/data/p253/p253_157.wav,Welsh,F,-356.328819,93.436145,-17.151477,22.421282,3.484256,-13.635613,-4.414813,...,-4.443295,9.623238,3.269109,0.699366,-5.421590,-6.574583,-0.331185,-2.762748,3.479336,-0.982944
2,/home/andriy/diploma/data/p253/p253_157.wav,Welsh,F,-282.376877,101.381353,3.877998,45.399120,8.878545,-12.205636,-7.659885,...,-10.836675,12.564437,-2.747082,-5.155890,-3.400530,-1.569530,-5.838708,-1.498904,0.338190,0.673177
3,/home/andriy/diploma/data/p253/p253_157.wav,Welsh,F,-674.073670,23.652606,4.435798,5.978649,4.340026,0.518874,0.819069,...,-0.839658,0.379472,-2.710150,0.808823,0.418095,-2.286779,-1.532727,-0.224329,-1.867465,-0.597333
4,/home/andriy/diploma/data/p253/p253_212.wav,Welsh,F,-245.017414,105.910309,-10.985298,33.220717,1.593071,-16.215820,-4.077472,...,-4.594752,10.076008,-5.074755,0.103556,-9.785228,-4.852409,-6.474127,-0.022581,-2.028267,-6.460203


In [3]:
train, other = train_test_split(df_accent_mfccs, test_size=0.2, random_state=42, shuffle=True)
dev, test = train_test_split(other, test_size=0.5, random_state=21, shuffle=True)
print(len(train), len(dev), len(test))

10376 1297 1298


In [4]:
X_test = test.drop('accent', axis=1) # setting up test set without 'ground truth'
y_train = train['accent'] # setting up y_train
X_train = train.drop('accent', axis=1)

In [5]:
dev[(dev.accent == 'NY')]

,accent,mfccs0,mfccs1,mfccs2,mfccs3,mfccs4,mfccs5,mfccs6,mfccs7,mfccs8,...,mfccs10,mfccs11,mfccs12,mfccs13,mfccs14,mfccs15,mfccs16,mfccs17,mfccs18,mfccs19
3473,NY,-315.684000,64.057711,11.377439,16.847353,11.260903,7.321917,-6.419385,-12.058146,-13.125508,...,-2.706626,1.355585,-1.632106,12.641259,-4.530686,-4.926789,-12.490177,9.865918,-4.825944,-0.445013
3785,NY,-249.983591,56.778504,17.730473,19.488555,14.532860,20.135691,-4.970480,1.838099,-7.577799,...,-4.255254,4.918007,0.476569,6.830578,2.606572,9.964627,-3.045495,5.032616,-2.974392,-0.814757
3488,NY,-335.964045,124.559715,-16.602319,29.681405,16.476776,-5.920455,-3.714903,-10.468281,-21.880210,...,8.459588,-1.487675,-4.613981,9.805680,-3.636208,-6.338505,-5.654107,6.897563,-3.024430,-2.330331
2684,NY,-347.140084,65.882821,10.532875,49.398033,33.848959,-6.854187,-3.470883,0.508582,-17.363390,...,-11.979601,5.815153,-1.598509,3.251070,-0.464801,2.296375,-10.169611,3.683526,-0.122838,3.475417
2901,NY,-285.799187,148.472243,-32.169172,13.709389,6.020103,-6.377009,7.605956,-5.828789,-6.214596,...,-2.803075,13.576090,-4.958839,0.794025,-1.813345,-3.828533,-13.908582,-2.264513,2.176093,-3.818351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192,NY,-671.087438,18.319633,12.732671,10.608754,6.210724,10.476397,-2.005354,3.658579,0.847810,...,-1.561119,3.490176,2.889945,2.288380,3.016351,5.638700,-0.354110,2.457570,-1.251288,0.862678
2884,NY,-336.325266,124.492555,-10.363928,45.119428,13.119256,-16.040718,9.205787,-20.220642,-17.490012,...,5.554061,-3.479373,-2.915867,9.569522,-3.258782,-2.471802,-8.761307,3.768359,-3.227055,-2.483960
2968,NY,-306.735686,111.010031,-6.130443,20.425648,17.454349,-11.105028,0.267787,-12.879790,-10.424200,...,0.617347,1.547278,-1.901814,6.000685,-5.848395,-10.216965,-11.342336,4.557373,-3.890743,-1.810216
3461,NY,-369.920710,118.827614,-2.133068,40.763423,17.986700,4.911760,-3.569136,-4.817815,-15.155285,...,3.323317,7.338819,-0.717833,6.776086,-1.902774,-8.192167,-5.178996,3.168638,2.393556,-3.148887


In [6]:
# dev_to_pred = dev.loc[4192, :].drop('accent', axis=0)
# dev_to_pred = np.array(dev_to_pred).reshape(1,-1)
dev_to_pred = pd.read_pickle('/home/andriy/diploma/CNN/cnn/australian_sample_trimmed.pkl')
dev_to_pred

,mfccs0,mfccs1,mfccs2,mfccs3,mfccs4,mfccs5,mfccs6,mfccs7,mfccs8,mfccs9,mfccs10,mfccs11,mfccs12,mfccs13,mfccs14,mfccs15,mfccs16,mfccs17,mfccs18,mfccs19
0,-430.947786,61.680485,-31.932617,79.238834,-20.183481,-22.802081,0.050954,-12.222050,-24.546997,7.738628,-25.128970,-1.218341,-22.287415,-6.668797,-2.746419,-13.947361,-4.576753,-7.791707,-6.151806,-0.807866
1,-700.472891,19.046094,8.391341,2.337616,-5.704956,-9.575362,-3.444111,-0.565775,-9.963947,3.860685,-13.879308,-3.032145,-4.325703,1.838993,-3.709635,9.904767,3.836077,-3.507716,-0.030687,-0.348879
2,-542.606415,98.640786,19.589038,28.438045,-1.711717,-26.072226,-7.082780,3.481938,-21.765297,12.479932,-19.851553,-7.500403,-13.241932,-7.316071,-17.957050,3.423629,-9.485463,-16.488090,4.995169,-4.455247
3,-514.336992,95.870917,1.815749,12.486512,-3.868825,-21.249552,-9.084166,-2.126001,-17.777816,13.386424,-19.057532,-10.118364,-12.505795,-8.976092,-21.864573,6.149273,-8.304119,-20.010422,5.155509,-0.851716
4,-464.492693,99.132136,-12.336098,30.018832,-4.950762,-29.572906,-22.692607,-0.775752,-3.700671,2.000574,-17.350186,-7.865829,-16.050680,-8.731781,-27.343332,-1.485498,-1.761753,-21.626753,-0.604266,0.714110
5,-450.280040,77.507627,-56.296523,8.472119,-21.588969,-9.140000,-12.908816,-1.019539,-4.828039,14.718502,-23.206544,-6.034941,-16.178328,5.779023,-14.467117,-8.920326,10.567294,-25.755446,2.342910,-6.440516
6,-580.249081,69.166109,-28.905479,9.281585,-23.142154,-6.736166,15.653108,-15.306484,-22.717843,11.364433,-13.297447,-14.818653,-10.019190,-0.916845,-12.732831,-3.045375,2.241149,-15.365500,-0.733274,1.373419
7,-462.685399,79.194345,5.093375,35.821860,-0.301105,-32.512059,-19.688003,-8.753459,-24.022346,-7.783893,-18.960032,-9.006542,-22.232131,-1.329811,-16.453609,-8.597465,0.784213,-21.009268,-4.490409,1.770492
8,-625.774212,109.112961,10.259371,-9.517081,-11.049075,-8.322182,-11.192811,-10.827096,-15.775046,7.542876,-11.555504,-10.663518,-10.505784,-5.006480,-8.638298,-4.497797,-4.641984,-8.630487,-0.496114,-6.265703
9,-485.059010,99.098618,14.170030,-10.881151,-25.807910,3.542105,-10.200437,-8.235600,-35.409751,19.895657,-18.127029,-22.313582,-6.314600,-8.634310,-11.445720,-4.851210,-8.531898,-9.544765,-4.087449,-3.042441


In [7]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
dev_to_pred = scaler.transform(dev_to_pred)
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
prediction = rf.predict(dev_to_pred)
print(Counter(prediction))

Counter({'Canadian': 10, 'English': 2, 'American': 1, 'Australian': 1})
